####  Case study 1 
#### Hackathon 
##### - Shantam Bhuraria 
##### - 0697


In [1]:
import pandas as pd 
import numpy as np
import os
import warnings
import seaborn as sns 
import matplotlib.pyplot as plt 
from sklearn.preprocessing import StandardScaler  
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.metrics import precision_recall_curve, roc_curve, auc, roc_auc_score, f1_score, plot_confusion_matrix, precision_score, recall_score
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, classification_report, roc_auc_score, roc_curve


In [2]:
%matplotlib inline
warnings.filterwarnings("ignore")
sns.set(style="whitegrid")

##### Getting the Data 

In [3]:
train = pd.read_csv('/home/shantam/Desktop/Supervised L1 30th July/Fraud_prediction_/train.csv')
test = pd.read_csv('/home/shantam/Desktop/Supervised L1 30th July/Fraud_prediction_/test.csv')
col_des=pd.read_csv('/home/shantam/Desktop/Supervised L1 30th July/Fraud_prediction_/column_Desc.csv')

In [4]:
train.shape

(50000, 24)

In [5]:
train.head()

,CLAIMNO,REPORTEDDT,LOSSDT,BIRTHDT,SICCD,CAUSEOFINJURY_FLTRCD,BODYPART_FLTRCD,RTRN_TOWORKIND,NATUREOFINJURYCD,LOSSSTATECD,...,CLMNT_LON,CLMNT_STATECD,INSUREDSTATECD,CLMNT_NA_cleaned_root,N_PRVDR_NAME_NONPHYS_cleaned_root,N_REFRING_PHYS_final_root,INSUREDNA_cleaned_root,N_PAYTO_NAME_cleaned_root,Prov_Name_All_final_root,TARGET
0,1,2016-10-12,2016-09-06,1987-07-27,5199,8.0,39.0,U,10.0,MO,...,-74.178010,NJ,MO,triquillagibbs,NaN,NaN,centricgroupllc,NaN,NaN,0.0
1,2,2016-10-12,2016-09-12,1977-04-29,5199,7.0,39.0,U,36.0,MA,...,-71.336869,MA,MO,jameschamberlain,NaN,NaN,centricgroupllc,NaN,NaN,0.0
2,3,2016-10-12,2016-09-23,1800-01-01,5199,6.0,39.0,U,52.0,MA,...,0.000000,MA,MO,donnathompson,NaN,NaN,centricgroupllc,NaN,NaN,0.0
3,4,2016-10-26,2016-10-07,NaN,8748,8.0,69.0,Y,10.0,PA,...,-74.571367,NJ,PA,wcadmin,NaN,NaN,2016adminhomeofficeclaims,NaN,NaN,0.0
4,5,2016-10-26,2016-10-07,NaN,8748,8.0,69.0,Y,10.0,PA,...,-74.570548,NJ,PA,wcadmin,NaN,NaN,2016adminwcscwc,NaN,NaN,0.0


In [6]:
train.describe()

,CLAIMNO,CAUSEOFINJURY_FLTRCD,BODYPART_FLTRCD,NATUREOFINJURYCD,INSUREDLAT,INSUREDLON,CLMNT_LAT,CLMNT_LON,TARGET
count,50000.000000,49981.000000,49984.000000,49984.000000,50000.000000,50000.000000,49751.000000,49751.000000,50000.000000
mean,25000.500000,5.851864,54.133603,44.493898,38.568179,-87.486055,37.426115,-86.774721,0.034520
std,14433.901067,2.245021,24.977218,24.090068,4.448390,15.512958,6.925168,19.668966,0.182562
min,1.000000,0.000000,10.000000,1.000000,0.000000,-157.917290,0.000000,-158.040920,0.000000
25%,12500.750000,4.000000,39.000000,28.000000,35.537616,-94.764500,34.131955,-95.839564,0.000000
50%,25000.500000,6.000000,50.000000,49.000000,40.229357,-83.258560,39.747290,-84.150673,0.000000
75%,37500.250000,8.000000,69.000000,52.000000,41.374330,-74.239419,41.279688,-74.912201,0.000000
max,50000.000000,11.000000,90.000000,91.000000,61.194220,0.000000,62.288830,0.000000,1.000000


In [7]:
# Check columns
train.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 24 columns):
CLAIMNO                              50000 non-null int64
REPORTEDDT                           50000 non-null object
LOSSDT                               50000 non-null object
BIRTHDT                              49816 non-null object
SICCD                                49988 non-null object
CAUSEOFINJURY_FLTRCD                 49981 non-null float64
BODYPART_FLTRCD                      49984 non-null float64
RTRN_TOWORKIND                       49999 non-null object
NATUREOFINJURYCD                     49984 non-null float64
LOSSSTATECD                          49556 non-null object
WORK_LOC_STATECD                     49934 non-null object
INSUREDLAT                           50000 non-null float64
INSUREDLON                           50000 non-null float64
CLMNT_LAT                            49751 non-null float64
CLMNT_LON                            49751 non-null float64


In [8]:
col_des

,Column,Description
0,CLAIMNO,Unique ID generated for each claim
1,REPORTEDDT,Date when the claim is reported
2,LOSSDT,Date when the loss has actually happened
3,BIRTHDT,Birth date of the claimant
4,SICCD,Industry code of the company
5,CAUSEOFINJURY_FLTRCD,Cause of Injury code
6,BODYPART_FLTRCD,Code for Body part where injury happened
7,RTRN_TOWORKIND,Has returned to work after the injury or not
8,NATUREOFINJURYCD,Code for the nature of Injury
9,LOSSSTATECD,Location where the loss has happened


In [9]:
# checking for duplicate values
dups=train.duplicated()
print('Number of duplicate rows = %d' % (dups.sum()))


Number of duplicate rows = 0


In [10]:
#checking for missing values 
print(train.isnull().values.any())
# Sum of missing values in different columns
print(train.isnull().sum())

True
CLAIMNO                                  0
REPORTEDDT                               0
LOSSDT                                   0
BIRTHDT                                184
SICCD                                   12
CAUSEOFINJURY_FLTRCD                    19
BODYPART_FLTRCD                         16
RTRN_TOWORKIND                           1
NATUREOFINJURYCD                        16
LOSSSTATECD                            444
WORK_LOC_STATECD                        66
INSUREDLAT                               0
INSUREDLON                               0
CLMNT_LAT                              249
CLMNT_LON                              249
CLMNT_STATECD                          647
INSUREDSTATECD                          29
CLMNT_NA_cleaned_root                  499
N_PRVDR_NAME_NONPHYS_cleaned_root    39229
N_REFRING_PHYS_final_root            40255
INSUREDNA_cleaned_root                  62
N_PAYTO_NAME_cleaned_root            16772
Prov_Name_All_final_root             25866
TARGET

In [11]:
per_mis = train.isnull().sum() * 100 / len(train)
data = {'percent_missing ': per_mis}
data

{'percent_missing ': CLAIMNO                               0.000
 REPORTEDDT                            0.000
 LOSSDT                                0.000
 BIRTHDT                               0.368
 SICCD                                 0.024
 CAUSEOFINJURY_FLTRCD                  0.038
 BODYPART_FLTRCD                       0.032
 RTRN_TOWORKIND                        0.002
 NATUREOFINJURYCD                      0.032
 LOSSSTATECD                           0.888
 WORK_LOC_STATECD                      0.132
 INSUREDLAT                            0.000
 INSUREDLON                            0.000
 CLMNT_LAT                             0.498
 CLMNT_LON                             0.498
 CLMNT_STATECD                         1.294
 INSUREDSTATECD                        0.058
 CLMNT_NA_cleaned_root                 0.998
 N_PRVDR_NAME_NONPHYS_cleaned_root    78.458
 N_REFRING_PHYS_final_root            80.510
 INSUREDNA_cleaned_root                0.124
 N_PAYTO_NAME_cleaned_root         

### Droping columns which is not needed in the classification 


In [12]:
train.drop(['CLAIMNO','REPORTEDDT','LOSSDT','N_PAYTO_NAME_cleaned_root','Prov_Name_All_final_root','INSUREDNA_cleaned_root',
                   'N_REFRING_PHYS_final_root','CLMNT_NA_cleaned_root','N_PRVDR_NAME_NONPHYS_cleaned_root'],axis=1,inplace=True)

test.drop(['CLAIMNO','REPORTEDDT','LOSSDT','N_PAYTO_NAME_cleaned_root','Prov_Name_All_final_root','INSUREDNA_cleaned_root',
                   'N_REFRING_PHYS_final_root','CLMNT_NA_cleaned_root','N_PRVDR_NAME_NONPHYS_cleaned_root'],axis=1,inplace=True)

In [13]:
train.head()

,BIRTHDT,SICCD,CAUSEOFINJURY_FLTRCD,BODYPART_FLTRCD,RTRN_TOWORKIND,NATUREOFINJURYCD,LOSSSTATECD,WORK_LOC_STATECD,INSUREDLAT,INSUREDLON,CLMNT_LAT,CLMNT_LON,CLMNT_STATECD,INSUREDSTATECD,TARGET
0,1987-07-27,5199,8.0,39.0,U,10.0,MO,MO,38.683566,-90.402551,40.744900,-74.178010,NJ,MO,0.0
1,1977-04-29,5199,7.0,39.0,U,36.0,MA,MO,38.683566,-90.402551,41.982333,-71.336869,MA,MO,0.0
2,1800-01-01,5199,6.0,39.0,U,52.0,MA,MO,38.683566,-90.402551,0.000000,0.000000,MA,MO,0.0
3,NaN,8748,8.0,69.0,Y,10.0,PA,PA,40.623995,-75.335088,40.645290,-74.571367,NJ,PA,0.0
4,NaN,8748,8.0,69.0,Y,10.0,PA,PA,40.623995,-75.335088,40.644471,-74.570548,NJ,PA,0.0


In [14]:
train.shape

(50000, 15)

In [15]:
# Adding age column
import datetime
train['BIRTHDT'] = pd.to_datetime(train['BIRTHDT'], format='%Y-%m-%d')
age= [(datetime.datetime.now()-i).days/365 for i in train['BIRTHDT']]
train['Age']=age

In [16]:
train['Age']

0         34.043836
1         44.293151
2        221.734247
3               NaN
4               NaN
            ...    
49995     39.032877
49996     24.567123
49997     48.186301
49998     35.126027
49999     24.536986
Name: Age, Length: 50000, dtype: float64

In [17]:
train['Age'].fillna(int(train['Age'].mean()),inplace=True)
train.drop('BIRTHDT',axis=1,inplace=True)

In [18]:
train.head()

,SICCD,CAUSEOFINJURY_FLTRCD,BODYPART_FLTRCD,RTRN_TOWORKIND,NATUREOFINJURYCD,LOSSSTATECD,WORK_LOC_STATECD,INSUREDLAT,INSUREDLON,CLMNT_LAT,CLMNT_LON,CLMNT_STATECD,INSUREDSTATECD,TARGET,Age
0,5199,8.0,39.0,U,10.0,MO,MO,38.683566,-90.402551,40.744900,-74.178010,NJ,MO,0.0,34.043836
1,5199,7.0,39.0,U,36.0,MA,MO,38.683566,-90.402551,41.982333,-71.336869,MA,MO,0.0,44.293151
2,5199,6.0,39.0,U,52.0,MA,MO,38.683566,-90.402551,0.000000,0.000000,MA,MO,0.0,221.734247
3,8748,8.0,69.0,Y,10.0,PA,PA,40.623995,-75.335088,40.645290,-74.571367,NJ,PA,0.0,51.000000
4,8748,8.0,69.0,Y,10.0,PA,PA,40.623995,-75.335088,40.644471,-74.570548,NJ,PA,0.0,51.000000


In [19]:
test['BIRTHDT'] = pd.to_datetime(test['BIRTHDT'], format='%Y-%m-%d')
age= [(datetime.datetime.now()-i).days/365 for i in test['BIRTHDT']]
test['Age']=age
test['Age'].fillna(int(test['Age'].mean()),inplace=True)
test.drop('BIRTHDT',axis=1,inplace=True)

In [20]:
test.head()

,SICCD,CAUSEOFINJURY_FLTRCD,BODYPART_FLTRCD,RTRN_TOWORKIND,NATUREOFINJURYCD,LOSSSTATECD,WORK_LOC_STATECD,INSUREDLAT,INSUREDLON,CLMNT_LAT,CLMNT_LON,CLMNT_STATECD,INSUREDSTATECD,Age
0,3714,6.0,69.0,Y,49.0,IN,IN,38.330756,-87.584599,38.353296,-87.562580,IN,IN,35.876712
1,3711,4.0,90.0,Y,52.0,CA,CA,33.872700,-117.226130,33.862960,-117.443510,CA,CA,46.926027
2,8221,10.0,69.0,U,37.0,AZ,AZ,47.659840,-122.317740,33.459290,-111.721570,AZ,WA,30.610959
3,7922,8.0,69.0,Y,10.0,NY,NY,40.758111,-73.989806,40.759766,-73.928279,NY,NY,44.063014
4,8111,4.0,90.0,Y,1.0,CA,CA,34.058220,-118.445980,33.836400,-118.324240,CA,CA,40.282192


In [21]:
train.describe()

,CAUSEOFINJURY_FLTRCD,BODYPART_FLTRCD,NATUREOFINJURYCD,INSUREDLAT,INSUREDLON,CLMNT_LAT,CLMNT_LON,TARGET,Age
count,49981.000000,49984.000000,49984.000000,50000.000000,50000.000000,49751.000000,49751.000000,50000.000000,50000.000000
mean,5.851864,54.133603,44.493898,38.568179,-87.486055,37.426115,-86.774721,0.034520,51.991510
std,2.245021,24.977218,24.090068,4.448390,15.512958,6.925168,19.668966,0.182562,31.787807
min,0.000000,10.000000,1.000000,0.000000,-157.917290,0.000000,-158.040920,0.000000,-174.767123
25%,4.000000,39.000000,28.000000,35.537616,-94.764500,34.131955,-95.839564,0.000000,35.008219
50%,6.000000,50.000000,49.000000,40.229357,-83.258560,39.747290,-84.150673,0.000000,47.887671
75%,8.000000,69.000000,52.000000,41.374330,-74.239419,41.279688,-74.912201,0.000000,59.570548
max,11.000000,90.000000,91.000000,61.194220,0.000000,62.288830,0.000000,1.000000,221.734247


In [22]:
train.shape

(50000, 15)

In [23]:
test.shape

(27995, 14)

In [26]:
train['SICCD']=train['SICCD'].replace('\t','nan')
test['SICCD']=test['SICCD'].replace('\t','nan')

In [27]:
# filling null values 
for i in train.columns:
    if(train[i].dtypes =='object'):
        train[i].fillna(train[i].mode()[0],inplace=True)
#         test[i].fillna(test[i].mode()[0],inplace=True)
        
    else:
        train[i].fillna(int(train[i].mean()),inplace=True)
#         test[i].fillna(int(test[i].mean()),inplace=True)

In [28]:
for i in test.columns:
    if(test[i].dtypes =='object'):
        test[i].fillna(test[i].mode()[0],inplace=True)
        
    else:
        test[i].fillna(int(test[i].mean()),inplace=True)

In [29]:
#check for missing data 
col=[]
vals=[]
for i in train.columns:
    col.append(i)
    vals.append(len(train[i].unique()))
per_mis = train.isnull().sum() * 100 / len(train)
data = {'percent_missing ': per_mis}
data


{'percent_missing ': SICCD                   0.0
 CAUSEOFINJURY_FLTRCD    0.0
 BODYPART_FLTRCD         0.0
 RTRN_TOWORKIND          0.0
 NATUREOFINJURYCD        0.0
 LOSSSTATECD             0.0
 WORK_LOC_STATECD        0.0
 INSUREDLAT              0.0
 INSUREDLON              0.0
 CLMNT_LAT               0.0
 CLMNT_LON               0.0
 CLMNT_STATECD           0.0
 INSUREDSTATECD          0.0
 TARGET                  0.0
 Age                     0.0
 dtype: float64}

####  Since we dont have any missing data now we can proceed for further

###  Making Target variable of Training data as our Y variable

In [30]:
y = train['TARGET']
train.drop(['TARGET'],axis=1,inplace=True) # droping the column from the final training data 

In [31]:
train.shape

(50000, 14)

In [32]:
test.shape

(27995, 14)

In [33]:
for i in train.columns:
    if (train[i].dtype=='object'):
        print(i)

SICCD
RTRN_TOWORKIND
LOSSSTATECD
WORK_LOC_STATECD
CLMNT_STATECD
INSUREDSTATECD


In [34]:
#

In [35]:
#train_dummy = pd.get_dummies(train_df[['SICCD','RTRN_TOWORKIND','LOSSSTATECD','WORK_LOC_STATECD','CLMNT_STATECD','INSUREDSTATECD']])
#test_dummy = pd.get_dummies(test_df[['SICCD','RTRN_TOWORKIND','LOSSSTATECD','WORK_LOC_STATECD','CLMNT_STATECD','INSUREDSTATECD']])

In [36]:
train['SICCD']

0        5199
1        5199
2        5199
3        8748
4        8748
         ... 
49995    6324
49996    3711
49997    6021
49998    6021
49999    5065
Name: SICCD, Length: 50000, dtype: object

In [37]:
train_dummy = pd.get_dummies(train[['SICCD','RTRN_TOWORKIND','LOSSSTATECD','WORK_LOC_STATECD','CLMNT_STATECD','INSUREDSTATECD']])
test_dummy = pd.get_dummies(test[['SICCD','RTRN_TOWORKIND','LOSSSTATECD','WORK_LOC_STATECD','CLMNT_STATECD','INSUREDSTATECD']])

In [38]:
train_dummy.head()

,SICCD_0134,SICCD_0181,SICCD_0191,SICCD_0211,SICCD_0212,SICCD_0213,SICCD_0241,SICCD_0272,SICCD_0722,SICCD_0723,...,INSUREDSTATECD_SD,INSUREDSTATECD_TN,INSUREDSTATECD_TX,INSUREDSTATECD_UT,INSUREDSTATECD_VA,INSUREDSTATECD_VT,INSUREDSTATECD_WA,INSUREDSTATECD_WI,INSUREDSTATECD_WV,INSUREDSTATECD_WY
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
test_dummy.head()

,SICCD_0134,SICCD_0181,SICCD_0211,SICCD_0272,SICCD_0742,SICCD_0752,SICCD_1311,SICCD_1389,SICCD_1422,SICCD_1521,...,INSUREDSTATECD_SC,INSUREDSTATECD_SD,INSUREDSTATECD_TN,INSUREDSTATECD_TX,INSUREDSTATECD_UT,INSUREDSTATECD_VA,INSUREDSTATECD_VT,INSUREDSTATECD_WA,INSUREDSTATECD_WI,INSUREDSTATECD_WV
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### As shown above there are missing data from the test file ie from the various SICCD might be missing in test and similarly other columns 

#### Hence we need to add the missing data in the test file to make sure we have consistancey 

In [40]:
data_missing = set(train_dummy.columns)-set(test_dummy.columns)

In [41]:
data_missing

{'CLMNT_STATECD_AE',
 'CLMNT_STATECD_Co',
 'CLMNT_STATECD_ES',
 'CLMNT_STATECD_GU',
 'CLMNT_STATECD_HI',
 'CLMNT_STATECD_Ka',
 'CLMNT_STATECD_ND',
 'CLMNT_STATECD_SA',
 'CLMNT_STATECD_SO',
 'INSUREDSTATECD_AK',
 'INSUREDSTATECD_FQ',
 'INSUREDSTATECD_NL',
 'INSUREDSTATECD_WY',
 'LOSSSTATECD_BO',
 'LOSSSTATECD_Ba',
 'LOSSSTATECD_CH',
 'LOSSSTATECD_GU',
 'LOSSSTATECD_Gu',
 'LOSSSTATECD_HO',
 'LOSSSTATECD_NL',
 'LOSSSTATECD_PR',
 'LOSSSTATECD_Q.',
 'LOSSSTATECD_SH',
 'LOSSSTATECD_TE',
 'LOSSSTATECD_TI',
 'SICCD_0191',
 'SICCD_0212',
 'SICCD_0213',
 'SICCD_0241',
 'SICCD_0722',
 'SICCD_0723',
 'SICCD_1021',
 'SICCD_1382',
 'SICCD_1474',
 'SICCD_1541',
 'SICCD_1711',
 'SICCD_1731',
 'SICCD_1751',
 'SICCD_1799',
 'SICCD_1819',
 'SICCD_2024',
 'SICCD_2033',
 'SICCD_2034',
 'SICCD_2035',
 'SICCD_2064',
 'SICCD_2085',
 'SICCD_2086',
 'SICCD_2096',
 'SICCD_2252',
 'SICCD_2261',
 'SICCD_2282',
 'SICCD_2311',
 'SICCD_2325',
 'SICCD_2329',
 'SICCD_2331',
 'SICCD_2341',
 'SICCD_2353',
 'SICCD_2361',


#### Adding the missing data and putting the value 0 for it 

In [42]:
for p in data_missing:
    test_dummy[p] = 0

In [43]:
test_dummy.shape

(27995, 847)

In [44]:
data_missing1 = set(test_dummy.columns)-set(train_dummy.columns)

In [45]:
data_missing1

{'CLMNT_STATECD_PE',
 'LOSSSTATECD_NS',
 'LOSSSTATECD_OS',
 'LOSSSTATECD_ZH',
 'SICCD_2299',
 'SICCD_2531',
 'SICCD_2591',
 'SICCD_3586',
 'SICCD_4619',
 'SICCD_5145',
 'WORK_LOC_STATECD_MT'}

In [46]:
for q in data_missing1:
    train_dummy[q] = 0

In [47]:
test_dummy.shape

(27995, 847)

In [48]:
train_dummy.shape

(50000, 847)

#### Since both the dataframes are having equal number of columns we can further proceed

In [49]:
print(train.shape)
print(test.shape)

(50000, 14)
(27995, 14)


#### removing all the columns having object data type and we have already created dummy for it 

In [50]:
train.drop(['SICCD','RTRN_TOWORKIND','LOSSSTATECD','WORK_LOC_STATECD','CLMNT_STATECD','INSUREDSTATECD'],axis=1,inplace=True)
test.drop(['SICCD','RTRN_TOWORKIND','LOSSSTATECD','WORK_LOC_STATECD','CLMNT_STATECD','INSUREDSTATECD'],axis=1,inplace=True)

In [51]:
final_train=pd.concat([train,train_dummy],axis=1)
final_test=pd.concat([test,test_dummy],axis=1)

In [52]:
print(final_train.shape)
print(final_test.shape)

(50000, 855)
(27995, 855)


#### Splitting the dataset 

In [53]:
x=final_train

In [54]:
X_train, X_test, y_train, y_test = train_test_split( x, y, test_size=0.4, random_state=0)

In [55]:
from imblearn.over_sampling import SMOTE

In [56]:
sm = SMOTE(random_state=27, sampling_strategy=1.0)


In [57]:
X_train, y_train = sm.fit_resample(X_train, y_train)

####  Trying Random Forest 

In [63]:
ob = RandomForestClassifier(random_state=100,class_weight='balanced') 
ob.fit(X_train, y_train) 
y_pred = ob.predict(X_test) 
    
ob.score(X_test, y_test)

0.96085

In [59]:
recall_score(y_test,y_pred)

0.16148148148148148

In [60]:
precision_score(y_test,y_pred)

0.3343558282208589

In [61]:
f1_score(y_test,y_pred)

0.21778221778221776

In [68]:
res_random = ob.predict_proba(final_test)[:,1]

In [64]:
test_df= pd.read_csv('/home/shantam/Desktop/Supervised L1 30th July/Fraud_prediction_/test.csv')


In [65]:
claim_no = test_df['CLAIMNO']

In [66]:
output = pd.DataFrame(columns=['CLAIMNO','TARGET'])

In [67]:
output['CLAIMNO'] = claim_no


In [69]:
output['TARGET'] = res_random

output.to_csv('random_forest.csv',index=False)

output

,CLAIMNO,TARGET
0,50001,0.75
1,50002,0.67
2,50003,0.57
3,50004,0.98
4,50005,0.68
...,...,...
27990,77991,0.60
27991,77992,0.69
27992,77993,0.56
27993,77994,0.64


In [71]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report




In [72]:
from sklearn.model_selection import GridSearchCV

In [73]:
baseline = GradientBoostingClassifier(learning_rate=0.1, n_estimators=100,max_depth=3, min_samples_split=2, min_samples_leaf=1, subsample=1,max_features='sqrt', random_state=10)
baseline.fit(X_train,y_train)

GradientBoostingClassifier(max_features='sqrt', random_state=10, subsample=1)

In [75]:
predictors=list(X_train)
feat_imp = pd.Series(baseline.feature_importances_, predictors).sort_values(ascending=False)
print('Accuracy of the GBM on test set: {:.3f}'.format(baseline.score(X_test, y_test)))
pred=baseline.predict(X_test)
print(classification_report(y_test, pred))

Accuracy of the GBM on test set: 0.871
              precision    recall  f1-score   support

         0.0       0.98      0.88      0.93     19325
         1.0       0.13      0.51      0.21       675

    accuracy                           0.87     20000
   macro avg       0.56      0.69      0.57     20000
weighted avg       0.95      0.87      0.91     20000



In [76]:
res_gbm = baseline.predict_proba(final_test)[:,1]

In [77]:
res_gbm

array([0.62505622, 0.70149784, 0.58799309, ..., 0.45302239, 0.66555254,
       0.30819425])

In [78]:
gbm_output = pd.DataFrame(columns=['CLAIMNO','TARGET'])


In [79]:
gbm_output['CLAIMNO'] = claim_no
gbm_output['TARGET'] = res_gbm

gbm_output.to_csv('gbm.csv',index=False)

gbm_output

,CLAIMNO,TARGET
0,50001,0.625056
1,50002,0.701498
2,50003,0.587993
3,50004,0.710957
4,50005,0.535126
...,...,...
27990,77991,0.629643
27991,77992,0.470004
27992,77993,0.453022
27993,77994,0.665553


In [80]:
# trying bagging and boosting techniue 